## **AML : Assignment 1, MDS201803**

**initialization**

In [0]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plot

from sklearn.model_selection import train_test_split


import sklearn.gaussian_process as gp
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass


from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.0.0


### **Import Bayesian Optimization Module by Thomas Huijskens**

**Source**: https://github.com/thuijskens/bayesian-optimization

In [0]:
from google.colab import files
src = list(files.upload().values())[0]
open('gp.py','wb').write(src)
import gp

%load gp.py
%run gp.py

Saving gp.py to gp.py


### **Import Fashion mnist Data Set**

In [0]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [0]:
train_images.shape, test_images.shape

((60000, 28, 28), (10000, 28, 28))

### **Feature Engineering**

In [0]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
len(class_names)

10

each pixel size is between 1 and 255, thus to scale them in [0,1] interval we divide each pixel by 255

In [0]:
train_images = train_images / 255.0

test_images = test_images / 255.0

### **Neural Network Classification with Bayesian Optimization**

In [0]:
activation_list = ['relu', 'sigmoid', 'softmax', 'selu', 'softplus', 'tanh', 'softsign', 'hard_sigmoid', 'exponential']

In [0]:
def layer_generator(nodes, act_func):
    layer = keras.layers.Dense(nodes, activation = activation_list[act_func])
    return layer

In [0]:
## Define loss function 

def loss_func(params):
    
    n1 = np.int(params[0])
    a1 = np.int(params[1])

    # we keep the number of hidden layers 1 and optimize its nodesize and choice of activation function
    
    model = keras.Sequential([
      keras.layers.Flatten(input_shape=(28, 28)),
      layer_generator(n1, a1), # the hidden layer with n1 nodes and a1 activation, our goal is to find optimum n1 & a1
      layer_generator(10, 2)   # the final layer is of fixed node size 10 (since the total # labels are 10)
      ])

    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    model.fit(train_images, train_labels, epochs=1, verbose=0)

    train_loss, train_acc = model.evaluate(train_images,  train_labels, verbose=0)

    return train_acc
  

In [0]:
bounds = np.array([[100, 150],[0,8]])
# we keep the bounds on node size : [100,150]
# we keep the bounds on activation function : [0,8] [Since there are 9 activation functions available in activation_list]

xp, yp = bayesian_optimisation(n_iters=10, sample_loss=loss_func, 
                               bounds=bounds,
                               n_pre_samples=2)


In [0]:
# The maximum is at:
xp_hat = np.round(xp[np.array(yp).argmax(), :])[0], activation_list[np.int(np.round(xp[np.array(yp).argmax(), :])[1])]
yp_hat = np.max(yp)

print("The optimum NN architecture : ", xp_hat)
print("Best possible train accuracy : ",yp_hat)

The optimum NN architecture :  (146.0, 'sigmoid')
Best possible train accuracy :  0.8689333
